In [27]:
# parquet-fromcsv -s core_r10m_c20_schema.txt -i core_r10m_c20.csv -o core_r10m_c20.parquet
# /mnt/arrow-rs/target/release/examples/filterscan core_r10m_c20.parquet 4 int point 511 0
# python3 ../../benchmark/generator_v2/filter_generator.py core_r10m_c20.csv core_r10m_c20_config.json ../../benchmark/generator_v2/filter_config/core_filter.yaml
# cargo build --package parquet --example filterscan --features="arrow async" --release 

## parquet cpp
# /mnt/arrow-private/cpp/out/build/openformat-release/release/filter_scan core_r10m_c20.parquet --batch-size=1024 --columns=18 --filter_idx=18 --filter_type=1 --type=2 --v1=0000rg67SI8 --v2=5e7WzM1Ny9LMiGrx4rZBPVDih0

In [1]:
import os
import sys
import datetime
import pathlib
import pandas as pd
dir_path = pathlib.Path(os.path.abspath('')).parent.resolve()
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

In [6]:
%time
# generate csv
file_base = 'core_r10m_c20_new'
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
os.system(f'python3 gen_workloads.py core 10000000 20 {file_base}')
os.system(f'cp -r core_r10m_c20_new {PROJ_SRC_DIR}/python/index')
os.system(f'cp {file_base}/gen_data/{file_base}.csv {PROJ_SRC_DIR}/python/data_gen')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
🤔 INFO: too many collisions in int_unique_list, 6366348 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 546319 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 73427 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 6312 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 7536 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 830938 values just use (max_value + 1)!
🤔 INFO: too many collisions in string_unique_list, generating time:156.1197190284729!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!


0

In [7]:
# generate parquet and orc
# parquet-fromcsv -s core_r10m_c20_schema.txt -i core_r10m_c20.csv -o core_r10m_c20.parquet -f tsv
# df = pd.read_csv('core_r10m_c20.csv', header=None, sep='\t')
# df.to_csv('core_r10m_c20.csv', index=False, header=False)
os.chdir(f'{PROJ_SRC_DIR}/python')
os.system(f'python3 scripts/orc_exp.py {file_base} orc_default')
os.chdir(f'{PROJ_SRC_DIR}/python/index')
os.system(f'parquet-fromcsv -s {file_base}/gen_data/{file_base}_arrow_schema.txt -i \
    {file_base}/gen_data/{file_base}.csv -o {file_base}.parquet')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
os.system(f'mv ../{file_base}{orc_name_suffix} ./{file_base}.orc')

loading config from ./experiments/orc_default.json ...
----Running experiment:----
SNAPPY_speed_0.8
write orc time (s): 37.18224501609802
loading config from /mnt/OpenFormat/python/experiments/orc_default.json ...


0

In [8]:
%%time
# generate filters
# config_name = '../../benchmark/generator_v2/filter_config/core_filter.yaml'
# config_name = 'filter_config_sort_point.yaml'
config_name = 'filter_config_sort_range.yaml'
# a parameter than force the generator to stick to one column
# we chose col 9 and 11 to test zone map
designated_idx = 9
# designated_idx =11 
# designated_idx = ''
os.system(f'rm -rf point*.csv range*.csv')
os.system(f'''python3 ../../benchmark/generator_v2/filter_generator.py \
    core_r10m_c20.csv core_r10m_c20_config.json {config_name} {designated_idx}''')
config_name_prefix = config_name.split('.')[0]
os.makedirs(f"{config_name_prefix}_{designated_idx}", exist_ok=True)
os.system(f'rm -rf {config_name_prefix}_{designated_idx}/*')
os.system(f"cp point*.csv {config_name_prefix}_{designated_idx}")
os.system(f"cp range*.csv {config_name_prefix}_{designated_idx}")

No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
No satisfied pointer filters
CPU times: user 5.05 ms, sys: 6.31 ms, total: 11.4 ms
Wall time: 38.4 s


cp: cannot stat 'point*.csv': No such file or directory


0

In [4]:
# compile binaries
# parquet
os.chdir(f'{HOME_DIR}/arrow-rs')
os.system('cargo build --package parquet --example filterscan --features="arrow async" --release')
pq_exec = f'{HOME_DIR}/arrow-rs/target/release/examples/filterscan'
os.system('cargo install --path parquet --features=cli')
# orc
os.chdir(f'{HOME_DIR}/orc')
os.system('/usr/bin/cmake --build /mnt/orc/build --config Release --target FilterExp -j 10 --')
orc_exec = f'{HOME_DIR}/orc/build/c++/test/FilterExp'
os.chdir(f'{HOME_DIR}/OpenFormat/python/index')

   Compiling parquet v31.0.0 (/mnt/arrow-rs/parquet)
  --> parquet/./examples/filterscan.rs:18:5
   |
18 | use arrow::util::pretty::print_batches;
   |     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
   |
   = note: `#[warn(unused_imports)]` on by default

  --> parquet/./examples/filterscan.rs:27:5
   |
27 | use std::process::Command;
   |     ^^^^^^^^^^^^^^^^^^^^^



[  8%] Built target zlib_ep
[ 25%] Built target protobuf_ep
[ 30%] Built target lz4_ep
[ 44%] Built target snappy_ep
[ 45%] Built target zstd_ep
[ 54%] Built target googletest_ep
[ 98%] Built target orc
[100%] Built target FilterExp


    Finished release [optimized] target(s) in 33.92s


In [3]:
%%time
# run point exps, deprecated
import yaml
import numpy as np
config = yaml.safe_load(open(config_name))
config_list = config['filter_config']

os.system('rm outputs/stats.json')
output_stats = {}
for i in range(3):
    for fmt in ['parquet', 'orc']:
        for config in config_list:
            expectation = config.get('selectivity_expectation')
            point_filters = pd.read_csv(f'point_{expectation}.csv')
            for proj_type in ['all', 'one']:
                for index, row in point_filters.iterrows():
                    val = np.int64(row['value']) if row['dtype'] == 'int' else row['value']
                    os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                    if fmt == 'parquet':
                        output_stats['time'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                            core_r10m_c20.parquet {row['col_idx']} {row['dtype']} point \
                                {val} 0 {proj_type}").read().split('\n')[0].split(' ')[-2]
                        output_stats['time_preload'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                            core_r10m_c20.parquet {row['col_idx']} {row['dtype']} point \
                                {val} 0 {proj_type}").read().split('\n')[0].split(' ')[-2]
                    else:
                        output_stats['time'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                        core_r10m_c20.orc f{row['col_idx']} {row['dtype']} point \
                            {val} 0 {proj_type}").read().split('\n')[0].split(' ')[-1])*1000
                        output_stats['time_preload'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                            core_r10m_c20.orc f{row['col_idx']} {row['dtype']} point \
                                {val} 0 {proj_type}").read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['format'] = fmt
                    output_stats['i'] = i
                    output_stats['proj_type'] = proj_type
                    output_stats['selectivity'] = expectation
                    parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('filter_exp', timestamp))

CPU times: user 1.39 s, sys: 3.93 s, total: 5.31 s
Wall time: 37min 26s


0

In [11]:
%%time
# run range exps, deprecated
import yaml
import numpy as np
config = yaml.safe_load(open(config_name))
config_list = config['filter_config']

os.system('rm outputs/stats.json')
output_stats = {}
for i in range(3):
    for fmt in ['parquet', 'orc']:
        for config in config_list:
            expectation = config.get('selectivity_expectation')
            range_filters = pd.read_csv(f'range_{expectation}.csv')
            for proj_type in ['all', 'one']:
                for index, row in range_filters.iterrows():
                    val1 = np.int64(row['val1']) if row['dtype'] == 'int' else row['val1']
                    val2 = np.int64(row['val2']) if row['dtype'] == 'int' else row['val2']
                    os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                    if fmt == 'parquet':
                        output_stats['time'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                            core_r10m_c20.parquet {row['col_idx']} {row['dtype']} range \
                                {val1} {val2} {proj_type}").read().split('\n')[0].split(' ')[-2]
                        output_stats['time_preload'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                            core_r10m_c20.parquet {row['col_idx']} {row['dtype']} range \
                                {val1} {val2} {proj_type}").read().split('\n')[0].split(' ')[-2]
                    else:
                        output_stats['time'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                        core_r10m_c20.orc f{row['col_idx']} {row['dtype']} range \
                            {val1} {val2} {proj_type}").read().split('\n')[0].split(' ')[-1])*1000
                        output_stats['time_preload'] = float(os.popen(f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                            core_r10m_c20.orc f{row['col_idx']} {row['dtype']} range \
                                {val1} {val2} {proj_type}").read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['format'] = fmt
                    output_stats['i'] = i
                    output_stats['proj_type'] = proj_type
                    output_stats['selectivity'] = expectation
                    parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('filter_range_sort', timestamp))

CPU times: user 1.64 s, sys: 5.48 s, total: 7.12 s
Wall time: 48min 57s


0

In [9]:
%%time
# select
import yaml
import numpy as np
config = yaml.safe_load(open(config_name))
config_list = config['filter_config']

os.system('rm outputs/stats.json')
output_stats = {}
for i in range(3):
    for fmt in ['parquet', 'orc']:
        for config in config_list:
            expectation = config.get('selectivity_expectation')
            try:
                range_filters = pd.read_csv(f'range_{expectation}.csv')
            except:
                continue
            for proj_type in ['one']:
                for index, row in range_filters.iterrows():
                    val1 = np.int64(row['val1']) if row['dtype'] == 'int' else row['val1']
                    val2 = np.int64(row['val2']) if row['dtype'] == 'int' else row['val2']
                    os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                    if fmt == 'parquet':
                        cmd = f"parquet-filterscan-noeval -f \
                            core_r10m_c20.parquet -i {row['col_idx']} -l {val1} -r \
                                {val2}"
                        output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                        output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                    else:
                        cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                        core_r10m_c20.orc f{row['col_idx']} {row['dtype']} range \
                            {val1} {val2} {proj_type}"
                        output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                        output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                    output_stats['format'] = fmt
                    output_stats['i'] = i
                    output_stats['proj_type'] = proj_type
                    output_stats['selectivity'] = expectation
                    parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format(f'filter_range_col{designated_idx}_eval', timestamp))

CPU times: user 1.07 s, sys: 3.79 s, total: 4.85 s
Wall time: 3min 13s


0

In [20]:
%%time
# run range exps, parquet select scan e2e
import yaml
import numpy as np
config = yaml.safe_load(open(config_name))
config_list = config['filter_config']

os.system('rm outputs/stats.json')
output_stats = {}
for i in range(3):
    output_stats['i'] = i
    for fmt in ['parquet']:
        output_stats['format'] = fmt
        for config in config_list:
            expectation = config.get('selectivity_expectation')
            try:
                range_filters = pd.read_csv(f'range_{expectation}.csv')
            except:
                continue
            for proj_type in ['all']:
                output_stats['proj_type'] = proj_type
            # val1 = 5.797
            # val2 = 34300092.4141
            for index, row in range_filters.iterrows():
                val1 = np.int64(row['val1']) if row['dtype'] == 'int' else row['val1']
                val2 = np.int64(row['val2']) if row['dtype'] == 'int' else row['val2']
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                output_stats['time'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                                    core_r10m_c20.parquet 11 float range \
                                        {val1} {val2} all").read().split('\n')[0].split(' ')[-2]
                output_stats['time_preload'] = os.popen(f"{HOME_DIR}/arrow-rs/target/release/examples/filterscan \
                    core_r10m_c20.parquet 11 float range \
                                        {val1} {val2} all").read().split('\n')[0].split(' ')[-2]
                output_stats['selectivity'] = expectation
                parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('filter_range_pq_e2e', timestamp))

CPU times: user 43.5 ms, sys: 26.3 ms, total: 69.8 ms
Wall time: 35.9 s


0